In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# check current directory
%pwd 

'C:\\Users\\Haiyan\\Google Drive\\DataScience\\Python\\Data_Scientist_Nanodegree\\1_Project1_Airbnb'

In [4]:
# set working directory
%cd "C:\Users\Haiyan\Google Drive\DataScience\Python\Data_Scientist_Nanodegree\1_Project1_Airbnb"

C:\Users\Haiyan\Google Drive\DataScience\Python\Data_Scientist_Nanodegree\1_Project1_Airbnb


In [ ]:
# read datasets
data = pd.read_csv('listings.csv')